In [1]:
import requests
import pandas as pd
import numpy as np
import uuid
from typing import List, Optional, Callable, Union
from utils.utils import wait
from app.app_handler import AppHandler
from demand_models.lgbm_model import LGBMModel
import json
import lightgbm as lgb
import os
from sklearn.metrics import mean_absolute_error
from price_models.canc_baseline_price import CancBaseline
from utils.utils import wait

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('configs/app_config.json') as json_file:
    config = json.load(json_file)

In [3]:
# UUID = uuid.uuid4().hex
# print(UUID)

In [4]:
# !echo -n $UUID > uuid.txt

In [5]:
UUID = open('uuid.txt').read()
UUID

'2718520d4ae740d48cec08212f4a816d'

In [6]:
app_handler = AppHandler(uuid=UUID,
                         url_begin='https://lab.karpov.courses/hardml-api/project-1/task/{}/begin'.format(UUID),
                         url_task_data_get='https://lab.karpov.courses/hardml-api/project-1/task/{}/data'\
                         .format(UUID),
                         url_task_result_get='https://lab.karpov.courses/hardml-api/project-1/task/{}/result'\
                         .format(UUID),
                         url_task_result_post='https://lab.karpov.courses/hardml-api/project-1/task/{}/result/'\
                         .format(UUID),
                         url_task_score_get='https://lab.karpov.courses/hardml-api/project-1/task/{}/lms_result'\
                         .format(UUID))

In [7]:
df_canc = pd.read_csv(config['canc_path'])
df_trans = pd.read_csv(config['transactions_path'])

# Baseline approach

In [12]:
canc_baseline_model = CancBaseline(strategy='min_related', coef=0.99)
canc_baseline_model.fit(df_canc, df_trans)

prev_score = 0

for batch_idx in range(30):
    if batch_idx == 0:
        app_handler.start()
    
    wait(5)
    
    df_batch = app_handler.get_batch()
    
    wait(5)
    
    df_pred = canc_baseline_model.predict(df_batch.copy())
    app_handler.send_batch_predictions(df_pred)
    
    wait(5)
    
    # df_batch_results = app_handler.get_batch_results()
    cur_score = app_handler.get_current_score()
    print('Batch: ', batch_idx, '\t Score: ', cur_score, '\t Diff. score: ', cur_score - prev_score)
    
    prev_score = cur_score

Start working:
{'status': 'ok'}
Batch:  0 	 Score:  1097612 	 Diff. score:  1097612
Batch:  1 	 Score:  2177620 	 Diff. score:  1080008
Batch:  2 	 Score:  3296721 	 Diff. score:  1119101
Batch:  3 	 Score:  4403339 	 Diff. score:  1106618
Batch:  4 	 Score:  5540237 	 Diff. score:  1136898
Batch:  5 	 Score:  6847434 	 Diff. score:  1307197
Batch:  6 	 Score:  7960715 	 Diff. score:  1113281
Batch:  7 	 Score:  9279443 	 Diff. score:  1318728
Batch:  8 	 Score:  10289117 	 Diff. score:  1009674
Batch:  9 	 Score:  11305067 	 Diff. score:  1015950
Batch:  10 	 Score:  12466159 	 Diff. score:  1161092
Batch:  11 	 Score:  13490753 	 Diff. score:  1024594
Batch:  12 	 Score:  14637248 	 Diff. score:  1146495
Batch:  13 	 Score:  15623033 	 Diff. score:  985785
Batch:  14 	 Score:  16818586 	 Diff. score:  1195553
Batch:  15 	 Score:  17844296 	 Diff. score:  1025710
Batch:  16 	 Score:  18927735 	 Diff. score:  1083439
Batch:  17 	 Score:  20397464 	 Diff. score:  1469729
Batch:  18 	 Sc

In [19]:
df = pd.read_csv('/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/sales_plan.csv')


3432     10060
3435     10121
3442     10216
3452     10230
3465     10388
         ...  
12157    98842
12169    99266
12181    99310
12199    99974
12211    99996
Name: SKU, Length: 826, dtype: int64

In [ ]:
# 